In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#Chargement des données
data = pd.read_csv('data/eucircos_regions_departements_circonscriptions_communes_gps.csv', sep=';')

#liste les communes et nettoyage
commune = data['nom_commune']
commune = commune.str.lower().str.replace(r'-|\'',' ')
commune = commune.str.lower().str.replace(r'é|è|ê|ë','e')
commune = commune.str.lower().str.replace(r'à|ä|â','a')
commune = commune.str.lower().str.replace(r'ï|î','i')
commune = commune.str.lower().str.replace(r'ö|ô','o')
commune = commune.str.lower().str.replace(r'ù|û|ü','u')
commune = commune.str.lower().str.replace('œ','oe')
commune = commune.str.lower().str.replace('æ','ae')
commune = commune.str.lower().str.replace(r'\(|\)',' ')
commune = commune.str.lower().str.replace(r'\s+',' ')
commune = ' ' + commune + ' '
commune = commune.str.lower().str.replace(r'(^|\s)(le|la|les|a|d|l|au|des|du|de|aux|en|un|une)($|\s)','')
commune = ' ' + commune + ' '
commune = commune.str.lower().str.replace(r'\s+',' ')

#liste des mots uniques distincts
mots = pd.Series(commune.str.cat(sep=' ')).str.split()
mots = pd.Series(mots[0])
mots = pd.Series(mots.sort_values().unique())

In [ ]:
#calcul du score
resultats = pd.DataFrame()
resultats['mot'] = mots
resultats['score'] = 0

# on boucle
for row in resultats.itertuples():
    score = commune.str.contains(' ' + row[1] + ' ').sum() - 1
    resultats.at[row[0], 'score'] = score
    if row[0]%1000 == 0:
        print(row[0])

# on stock temporairement
resultats
resultats = resultats.sort_values('score', ascending=False)
resultats.to_csv('result.csv', index=False)

# donc on a une liste des mots avec le nombre de commune qui le contienne

In [ ]:
# chargement du résultat intermédiaire
resultats = pd.read_csv('result.csv', sep=",")

# a partir de là, il faut prendre le premier mot, compter le nombre de commune que ça concerne, les enlever de la liste et recommencer avec le second mot et ainsi de suite
startList = commune
stopIndex = 100
perf = pd.DataFrame(columns=['mot', 'contains', 'restant'])
perf['mot'] = resultats['mot']
for row in resultats.itertuples() :
    #if row[0] <= stopIndex :
    contains = startList[startList.str.contains(' ' + str(row[1]) + ' ')]
    startList = startList.drop(contains.index)
    perf.at[row[0], 'contains'] = contains.size
    perf.at[row[0], 'restant'] = startList.size
    if row[0]%1000 == 0 :
        print(str(row[0]) + " : " + str(row[1]) + " - " + str(contains.size) + " - " + str(startList.size))
    
perf.to_csv('perf.csv', sep=",")